In [1]:
import os
os.chdir('..')

In [41]:
import torch
import numpy as np
from util.misc import assert_shape

In [4]:
pred = np.load('pred.npy')
pred2 = np.load('pred2.npy')

In [6]:
pred.shape, pred2.shape

((20899, 17, 3), (20899, 17, 3))

In [9]:
np.all(pred == pred2)

True

# combine_pose_and_trans

In [17]:
d = np.load('combine_pose_and_trans.npz', allow_pickle=True)

In [55]:
data3d=d['data3d']
std3d=d['std3d']
mean3d=d['mean3d']
joint_set=d['joint_set'][()]
root_name=d['root_name'][()]
log_root_z=d['log_root_z'][()]

In [56]:
data3d = torch.from_numpy(data3d)

In [57]:
assert_shape(data3d, (None, joint_set.NUM_JOINTS * 3))

data3d = data3d * std3d + mean3d
root = data3d[:, -3:]  # (201, 3)
rel_pose = data3d[:, :-3].reshape((len(data3d), joint_set.NUM_JOINTS - 1, 3))  # (201, 16, 3)

if log_root_z:
    root[:, 2] = torch.exp(root[:, 2])

rel_pose += root[:, None, :]

result = torch.zeros((len(data3d), joint_set.NUM_JOINTS, 3))
root_ind = joint_set.index_of(root_name)
result[:, :root_ind, :] = rel_pose[:, :root_ind, :]
result[:, root_ind, :] = root
result[:, root_ind + 1:, :] = rel_pose[:, root_ind:, :]

In [64]:
type(data3d) == torch.Tensor

True

# _group_by_seq

In [77]:
data = np.load('_group_by_seq.npy', allow_pickle=True)[()]

In [78]:
data = {k: torch.from_numpy(v) for k, v in data.items()}

In [79]:

per_person_keys = sorted(data.keys())
result = {}
for seq in range(1, 21):
    keys = sorted([k for k in per_person_keys if k.startswith('%d/' % seq)])
    assert len(keys) > 0, per_person_keys
    if type(data[keys[0]]) == torch.Tensor:
        result[seq] = torch.cat([data[k] for k in keys])
    else:
        result[seq] = np.concatenate([data[k] for k in keys])

In [ ]:

{k: v.cpu().numpy()for k, v in result.items()}

# pose_ref

In [83]:
d = np.load('pose_ref.npz')

In [91]:
total_loss = d['total_loss'][()]
pose_loss = d['pose_loss'][()]
velocity_loss_hip = d['velocity_loss_hip'][()]
velocity_loss_hip_large = d['velocity_loss_hip_large'][()]
velocity_loss_rel = d['velocity_loss_rel'][()]
velocity_loss_rel_large = d['velocity_loss_rel_large'][()]

In [92]:
{'total_loss': d['total_loss'][()],
'pose_loss': d['pose_loss'][()],
'velocity_loss_hip': d['velocity_loss_hip'][()],
'velocity_loss_hip_large': d['velocity_loss_hip_large'][()],
'velocity_loss_rel': d['velocity_loss_rel'][()],
'velocity_loss_rel_large': d['velocity_loss_rel_large'][()]}

{'total_loss': 0.456771,
 'pose_loss': 0.0,
 'velocity_loss_hip': 0.11577673,
 'velocity_loss_hip_large': 2.1961489,
 'velocity_loss_rel': 0.61452806,
 'velocity_loss_rel_large': 5.9926534}